### On the 11th of July, 2020, I started an NLP competition on Kaggle. This is part of my topdown approach to learning Natural Language Processing

**In this competition, one’s is challenged to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t**

*Often, I collect the datasets & store in my local machine, look for a notebook in the Kaggle notebook section, and use it as a guide. I read each cell and research on why the author used the codes in each cell.*

Link to the Kaggle Notebook: https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert

**Getting Started**

*import modules*

In [2]:
import gc # provides an interface to the optional garbage collector.
# The process by which Python periodically frees and reclaims blocks 
#of memory that no longer are in use is called Garbage Collection
import re # This module provides regular expression matching operations
import string # This module contains a number of functions to process standard Python strings
import operator # The operator module exports a set of efficient functions corresponding to the 
#intrinsic operators of Python.
from collections import defaultdict # This module implements specialized container datatypes 
#providing alternatives to Python’s general purpose built-in containers, dict, list, set, and tuple

import numpy as np # Numerical python
import pandas as pd # Data Analysis

# lets you customize some aspects of its behaviour, display-related options being those the user
#is most likely to adjust
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Visualization modules
import matplotlib.pyplot as plt 
import seaborn as sns


import tokenization # A general purpose text tokenizing module for python
from wordcloud import STOPWORDS # Module with common stop words

# Provides train/test indices to split data in train/test sets
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

# An open source software library for high performance numerical computation
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

SEED = 1337